## Comment
This script only runs the diphophate and duplicate test of the MACAW test suite.  
The output are csv files saved in save_dir. It will save these files as "model_id_phase_1.csv".  
So if your model filename was "my_model.xml" the results will be saved as "my_model_phase_1.csv"

## Imports

In [1]:
import cobra
from cobra.io import read_sbml_model
from macaw.main import *
import os

## Paths

In [2]:
# Path to your models
model_dir = '/home/lisa/Dokumente/Programmierung/Models/03_charge_balance/'

# Directory in which you want to save the resulting csv files
save_dir = '/home/lisa/Dokumente/Programmierung/Scripts/Frowin/macaw_csv/'

## Redox_pairs, proton_ids, phosphate_ids

In [8]:
redox_pairs = [
    # NAD(H), NADP(H), FAD(H2), and FMN(H2)
    ('nad_c', 'nadh_c'), ('nadp_c', 'nadph_c'), ('fad_c', 'fadh2_c'),
    ('fmn_c', 'fmnh2_c'),
    # ubiquinone-8 and ubiquinol-8
    ('q8_c', 'q8h2_c'),
    # menaquinone-8 and menaquinol-8
    ('mqn8_c', 'mql8_c'),
    # riboflavin
    ('ribflv_c', 'rbflvrd_c'),
    # glutathione
    ('gthox_c', 'gthrd_c'),
    # glutaredoxin
    ('grxox_c', 'grxrd_c'),
    # thioredoxin
    ('trdox_c', 'trdrd_c'),
    # flavodoxin (reduced and semi-oxidized)
    ('flxso_c', 'flxr_c'),
    # oxygen and hydrogen peroxide
    ('o2_c', 'h2o2_c')
]

In [10]:
def filter_existing_redox_pairs(model, redox_pairs):
    metabolites = {met.id for met in model.metabolites}
    return [pair for pair in redox_pairs if pair[0] in metabolites and pair[1] in metabolites]

In [4]:
# protons in all compartments
proton_ids = ['h_c', 'h_e', 'h_p']

In [5]:
# Phosphate in all compartments
pi_ids=["pi_c", "pi_p", "pi_e"]
ppi_ids=["ppi_c", "ppi_p", "ppi_e"]

## Main

In [11]:
for file in os.listdir(model_dir):

    model = read_sbml_model(os.path.join(model_dir, file))
    model.solver = "cplex"

    (diphosphate_results) = diphosphate_test(model, ppi_ids, pi_ids)

    # not every redox pair is in every model, so we filter first which redox pairs we actually have
    filtered_pairs = filter_existing_redox_pairs(model, redox_pairs)
    (duplicate_results, duplicate_edge_list) = duplicate_test(model, filtered_pairs, proton_ids)

    phase_1_results = duplicate_results.merge(diphosphate_results)

    phase_1_results.to_csv(os.path.join(save_dir, file.replace('.xml', '_phase_1.csv')), index=False)
    
    print(f"Finished processing {file}")

Starting diphosphate test...
 - Found 3 suspiciously reversible reactions that involve diphosphate.
Starting duplicate test...
 - Found 251 reactions that were some type of duplicate:
   - 4 were completely identical to at least one other reaction.
   - 50 involve the same metabolites but go in the opposite direction or have the opposite reversibility as at least one other reaction.
   - 110 involve the same metabolites but with different coefficients as at least one other reaction.
   - 135 redox reactions oxidize and/or redox the same metabolites as another redox reaction but use a different pair of the given redox carriers.
Finished processing AA4_curated.xml
Starting diphosphate test...
 - Found 7 suspiciously reversible reactions that involve diphosphate.
Starting duplicate test...
 - Found 274 reactions that were some type of duplicate:
   - 44 were completely identical to at least one other reaction.
   - 55 involve the same metabolites but go in the opposite direction or have t